# Tier 6: Advanced Anomaly Detection & Outlier Analysis

---

**Author:** Brandon Deloatch
**Affiliation:** Quipu Research Labs, LLC
**Date:** 2025-10-02
**Version:** v1.3
**License:** MIT
**Notebook ID:** f8bf2d16-b5d6-4a48-97f4-1d58238ec1c7

---

## Citation
Brandon Deloatch, "Tier 6: Advanced Anomaly Detection & Outlier Analysis," Quipu Research Labs, LLC, v1.3, 2025-10-02.

Please cite this notebook if used or adapted in publications, presentations, or derivative work.

---

## Contributors / Acknowledgments
- **Primary Author:** Brandon Deloatch (Quipu Research Labs, LLC)
- **Institutional Support:** Quipu Research Labs, LLC - Advanced Analytics Division
- **Technical Framework:** Built on scikit-learn, pandas, numpy, and plotly ecosystems
- **Methodological Foundation:** Statistical learning principles and modern data science best practices

---

## Version History
| Version | Date | Notes |
|---------|------|-------|
| v1.3 | 2025-10-02 | Enhanced professional formatting, comprehensive documentation, interactive visualizations |
| v1.2 | 2024-09-15 | Updated analysis methods, improved data generation algorithms |
| v1.0 | 2024-06-10 | Initial release with core analytical framework |

---

## Environment Dependencies
- **Python:** 3.8+
- **Core Libraries:** pandas 2.0+, numpy 1.24+, scikit-learn 1.3+
- **Visualization:** plotly 5.0+, matplotlib 3.7+
- **Statistical:** scipy 1.10+, statsmodels 0.14+
- **Development:** jupyter-lab 4.0+, ipywidgets 8.0+

> **Reproducibility Note:** Use requirements.txt or environment.yml for exact dependency matching.

---

## Data Provenance
| Dataset | Source | License | Notes |
|---------|--------|---------|-------|
| Synthetic Data | Generated in-notebook | MIT | Custom algorithms for realistic simulation |
| Statistical Distributions | NumPy/SciPy | BSD-3-Clause | Standard library implementations |
| ML Algorithms | Scikit-learn | BSD-3-Clause | Industry-standard implementations |
| Visualization Schemas | Plotly | MIT | Interactive dashboard frameworks |

---

## Execution Provenance Logs
- **Created:** 2025-10-02
- **Notebook ID:** f8bf2d16-b5d6-4a48-97f4-1d58238ec1c7
- **Execution Environment:** Jupyter Lab / VS Code
- **Computational Requirements:** Standard laptop/workstation (2GB+ RAM recommended)

> **Auto-tracking:** Execution metadata can be programmatically captured for reproducibility.

---

## Disclaimer & Responsible Use
This notebook is provided "as-is" for educational, research, and professional development purposes. Users assume full responsibility for any results, applications, or decisions derived from this analysis.

**Professional Standards:**
- Validate all results against domain expertise and additional data sources
- Respect licensing and attribution requirements for all dependencies
- Follow ethical guidelines for data analysis and algorithmic decision-making
- Credit all methodological sources and derivative frameworks appropriately

**Academic & Commercial Use:**
- Permitted under MIT license with proper attribution
- Suitable for educational curriculum and professional training
- Appropriate for commercial adaptation with citation requirements
- Recommended for reproducible research and transparent analytics

---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import DBSCAN
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print(" Tier 6: Advanced Anomaly Detection & Outlier Analysis")
print("=" * 55)
print(" CROSS-REFERENCES:")
print("• Prerequisites: Tier1_Descriptive.ipynb, Tier2_LogisticRegression.ipynb, Tier5_Classification.ipynb")
print("• Builds On: Tier4_SVM.ipynb (one-class SVM), Tier4_KMeans.ipynb (clustering-based)")
print("• Complements: Tier6_RealTimeAnalytics.ipynb, Tier3_TimeSeries.ipynb")
print("• Advanced: Tier6_DeepLearning.ipynb, Tier6_EnsembleMethods.ipynb")
print("=" * 55)
print("Anomaly Detection Techniques:")
print("• Statistical methods (Z-score, IQR, Grubbs' test)")
print("• Machine learning approaches (Isolation Forest, One-Class SVM)")
print("• Clustering-based detection (DBSCAN, LOF)")
print("• Ensemble methods and real-time detection")
print("• Performance evaluation and business impact analysis")

In [ ]:
# Generate comprehensive synthetic datasets with known anomalies
np.random.seed(42)

def generate_anomaly_datasets():
 """Generate multiple datasets with different types of anomalies."""

 # Dataset 1: Financial Transactions (fraud detection)
 n_normal = 8000
 n_fraud = 200

 # Normal transactions
 normal_amounts = np.random.lognormal(mean=3, sigma=1, size=n_normal)
 normal_amounts = np.clip(normal_amounts, 1, 1000) # Realistic transaction amounts

 normal_features = {
 'transaction_amount': normal_amounts,
 'account_age_days': np.random.normal(365, 200, n_normal),
 'daily_transaction_count': np.random.poisson(3, n_normal),
 'time_since_last_transaction': np.random.exponential(2, n_normal),
 'merchant_risk_score': np.random.beta(2, 5, n_normal),
 'is_weekend': np.random.choice([0, 1], n_normal, p=[0.7, 0.3]),
 'customer_id': np.random.randint(1, 2000, n_normal)
 }

 # Fraudulent transactions (anomalies)
 fraud_amounts = np.random.choice([
 np.random.uniform(2000, 10000, n_fraud//3), # High-value fraud
 np.random.uniform(0.1, 5, n_fraud//3), # Micro-fraud
 np.random.lognormal(5, 0.5, n_fraud//3) # Random large amounts
 ]).flatten()[:n_fraud]

 fraud_features = {
 'transaction_amount': fraud_amounts,
 'account_age_days': np.random.choice([
 np.random.uniform(0, 30, n_fraud//2), # New accounts
 np.random.uniform(1000, 2000, n_fraud//2) # Old dormant accounts
 ]).flatten()[:n_fraud],
 'daily_transaction_count': np.random.choice([
 np.random.poisson(15, n_fraud//2), # High frequency
 np.random.poisson(0.5, n_fraud//2) # Very low frequency
 ]).flatten()[:n_fraud],
 'time_since_last_transaction': np.random.exponential(0.1, n_fraud), # Rapid succession
 'merchant_risk_score': np.random.beta(5, 2, n_fraud), # Higher risk merchants
 'is_weekend': np.random.choice([0, 1], n_fraud, p=[0.4, 0.6]), # More weekend activity
 'customer_id': np.random.randint(1, 2000, n_fraud)
 }

 # Combine normal and fraudulent transactions
 financial_df = pd.DataFrame({
 key: np.concatenate([normal_features[key], fraud_features[key]])
 for key in normal_features.keys()
 })
 financial_df['is_fraud'] = np.concatenate([np.zeros(n_normal), np.ones(n_fraud)])
 financial_df = financial_df.sample(frac=1).reset_index(drop=True) # Shuffle

 # Dataset 2: Manufacturing Quality Control
 n_normal_parts = 5000
 n_defective = 150

 # Normal parts (multivariate normal distribution)
 normal_temp = np.random.normal(250, 10, n_normal_parts)
 normal_pressure = np.random.normal(15, 2, n_normal_parts)
 normal_vibration = np.random.normal(0.5, 0.1, n_normal_parts)
 normal_thickness = np.random.normal(5.0, 0.1, n_normal_parts)

 # Defective parts (various anomaly patterns)
 defective_temp = np.concatenate([
 np.random.normal(300, 15, n_defective//3), # Overheating
 np.random.normal(200, 10, n_defective//3), # Underheating
 np.random.normal(250, 50, n_defective//3) # Unstable temperature
 ])[:n_defective]

 defective_pressure = np.concatenate([
 np.random.normal(25, 5, n_defective//2), # High pressure
 np.random.normal(5, 2, n_defective//2) # Low pressure
 ])[:n_defective]

 defective_vibration = np.random.uniform(1.0, 3.0, n_defective) # High vibration
 defective_thickness = np.concatenate([
 np.random.normal(5.5, 0.2, n_defective//2), # Too thick
 np.random.normal(4.5, 0.2, n_defective//2) # Too thin
 ])[:n_defective]

 manufacturing_df = pd.DataFrame({
 'temperature': np.concatenate([normal_temp, defective_temp]),
 'pressure': np.concatenate([normal_pressure, defective_pressure]),
 'vibration': np.concatenate([normal_vibration, defective_vibration]),
 'thickness': np.concatenate([normal_thickness, defective_thickness]),
 'is_defective': np.concatenate([np.zeros(n_normal_parts), np.ones(n_defective)])
 })
 manufacturing_df = manufacturing_df.sample(frac=1).reset_index(drop=True)

 return financial_df, manufacturing_df

# Generate datasets
financial_df, manufacturing_df = generate_anomaly_datasets()

print(" Anomaly Detection Datasets Created:")
print(f"\n1. Financial Transactions Dataset:")
print(f" • Total transactions: {len(financial_df):,}")
print(f" • Fraudulent transactions: {financial_df['is_fraud'].sum():.0f} ({financial_df['is_fraud'].mean()*100:.1f}%)")
print(f" • Average transaction: ${financial_df['transaction_amount'].mean():.2f}")
print(f" • Fraud rate: 1 in {len(financial_df) / financial_df['is_fraud'].sum():.0f} transactions")

print(f"\n2. Manufacturing Quality Control:")
print(f" • Total parts inspected: {len(manufacturing_df):,}")
print(f" • Defective parts: {manufacturing_df['is_defective'].sum():.0f} ({manufacturing_df['is_defective'].mean()*100:.1f}%)")
print(f" • Average temperature: {manufacturing_df['temperature'].mean():.1f}°C")
print(f" • Quality rate: {(1-manufacturing_df['is_defective'].mean())*100:.1f}%")

# Display sample data
print(f"\nSample Financial Data:")
print(financial_df.head())

In [ ]:
# 1. STATISTICAL ANOMALY DETECTION METHODS
print(" 1. STATISTICAL ANOMALY DETECTION METHODS")
print("=" * 43)

def statistical_anomaly_detection(data, column, method='zscore', threshold=3):
 """Apply various statistical methods for anomaly detection."""

 if method == 'zscore':
 z_scores = np.abs(stats.zscore(data[column]))
 anomalies = z_scores > threshold
 scores = z_scores

 elif method == 'iqr':
 Q1 = data[column].quantile(0.25)
 Q3 = data[column].quantile(0.75)
 IQR = Q3 - Q1
 lower_bound = Q1 - 1.5 * IQR
 upper_bound = Q3 + 1.5 * IQR
 anomalies = (data[column] < lower_bound) | (data[column] > upper_bound)
 scores = np.abs(data[column] - data[column].median()) / IQR

 elif method == 'grubbs':
 # Simplified Grubbs test (single outlier)
 mean_val = data[column].mean()
 std_val = data[column].std()
 z_scores = np.abs((data[column] - mean_val) / std_val)

 n = len(data)
 t_critical = stats.t.ppf(1 - 0.05/(2*n), n-2) # Two-tailed test
 grubbs_critical = ((n-1)/np.sqrt(n)) * np.sqrt(t_critical**2/(n-2+t_critical**2))

 anomalies = z_scores > grubbs_critical
 scores = z_scores

 elif method == 'modified_zscore':
 # Using median absolute deviation (more robust)
 median = data[column].median()
 mad = np.median(np.abs(data[column] - median))
 modified_z_scores = 0.6745 * (data[column] - median) / mad
 anomalies = np.abs(modified_z_scores) > threshold
 scores = np.abs(modified_z_scores)

 return anomalies, scores

# Apply statistical methods to financial transaction amounts
print("Financial Transaction Amount Analysis:")

stat_methods = ['zscore', 'iqr', 'grubbs', 'modified_zscore']
stat_results = {}

for method in stat_methods:
 threshold = 3 if method in ['zscore', 'modified_zscore'] else None
 anomalies, scores = statistical_anomaly_detection(
 financial_df, 'transaction_amount', method, threshold
 )

 stat_results[method] = {
 'anomalies': anomalies,
 'scores': scores,
 'count': anomalies.sum(),
 'percentage': anomalies.mean() * 100
 }

 # Calculate precision, recall, F1 for fraud detection
 if 'is_fraud' in financial_df.columns:
 true_positives = ((anomalies) & (financial_df['is_fraud'] == 1)).sum()
 false_positives = ((anomalies) & (financial_df['is_fraud'] == 0)).sum()
 false_negatives = ((~anomalies) & (financial_df['is_fraud'] == 1)).sum()

 precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
 recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
 f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

 stat_results[method].update({
 'precision': precision,
 'recall': recall,
 'f1': f1
 })

 print(f"\n{method.upper()} Method:")
 print(f" • Anomalies detected: {anomalies.sum()} ({anomalies.mean()*100:.1f}%)")
 if 'precision' in stat_results[method]:
 print(f" • Precision: {precision:.3f}")
 print(f" • Recall: {recall:.3f}")
 print(f" • F1-Score: {f1:.3f}")

# Find best statistical method
best_stat_method = max(stat_methods, key=lambda x: stat_results[x].get('f1', 0))
print(f"\n Best Statistical Method: {best_stat_method.upper()} (F1: {stat_results[best_stat_method]['f1']:.3f})")

In [ ]:
# 2. MACHINE LEARNING ANOMALY DETECTION
print(" 2. MACHINE LEARNING ANOMALY DETECTION")
print("=" * 41)

# Prepare data for ML models
scaler = StandardScaler()

# Financial data preparation
financial_features = ['transaction_amount', 'account_age_days', 'daily_transaction_count',
 'time_since_last_transaction', 'merchant_risk_score', 'is_weekend']
X_financial = financial_df[financial_features].copy()
X_financial_scaled = scaler.fit_transform(X_financial)

# Manufacturing data preparation
manufacturing_features = ['temperature', 'pressure', 'vibration', 'thickness']
X_manufacturing = manufacturing_df[manufacturing_features].copy()
X_manufacturing_scaled = StandardScaler().fit_transform(X_manufacturing)

# Initialize ML anomaly detection models
ml_models = {
 'Isolation Forest': IsolationForest(contamination=0.1, random_state=42),
 'One-Class SVM': OneClassSVM(gamma='scale', nu=0.1),
 'Local Outlier Factor': LocalOutlierFactor(contamination=0.1),
 'Elliptic Envelope': EllipticEnvelope(contamination=0.1),
 'DBSCAN': DBSCAN(eps=0.5, min_samples=5)
}

def evaluate_ml_anomaly_detection(X, y_true, models, dataset_name):
 """Evaluate multiple ML anomaly detection models."""

 print(f"\n{dataset_name} Dataset Evaluation:")
 print("=" * (len(dataset_name) + 20))

 results = {}

 for model_name, model in models.items():
 try:
 if model_name == 'Local Outlier Factor':
 # LOF returns 1 for inliers, -1 for outliers
 predictions = model.fit_predict(X)
 anomaly_mask = predictions == -1

 elif model_name == 'DBSCAN':
 # DBSCAN labels: -1 for noise (anomalies), 0+ for clusters
 predictions = model.fit_predict(X)
 anomaly_mask = predictions == -1

 else:
 # Standard sklearn anomaly detection interface
 model.fit(X)
 predictions = model.predict(X)
 anomaly_mask = predictions == -1

 # Calculate performance metrics
 if y_true is not None:
 true_positives = ((anomaly_mask) & (y_true == 1)).sum()
 false_positives = ((anomaly_mask) & (y_true == 0)).sum()
 false_negatives = ((~anomaly_mask) & (y_true == 1)).sum()
 true_negatives = ((~anomaly_mask) & (y_true == 0)).sum()

 precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
 recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
 f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

 # Calculate AUC if possible
 try:
 if hasattr(model, 'decision_function'):
 scores = model.decision_function(X)
 auc = roc_auc_score(y_true, -scores) # Negative because anomalies have lower scores
 else:
 auc = None
 except:
 auc = None
 else:
 precision = recall = f1 = auc = None

 results[model_name] = {
 'anomaly_mask': anomaly_mask,
 'anomaly_count': anomaly_mask.sum(),
 'anomaly_percentage': anomaly_mask.mean() * 100,
 'precision': precision,
 'recall': recall,
 'f1': f1,
 'auc': auc,
 'model': model
 }

 print(f"\n{model_name}:")
 print(f" • Anomalies detected: {anomaly_mask.sum()} ({anomaly_mask.mean()*100:.1f}%)")
 if precision is not None:
 print(f" • Precision: {precision:.3f}")
 print(f" • Recall: {recall:.3f}")
 print(f" • F1-Score: {f1:.3f}")
 if auc is not None:
 print(f" • AUC: {auc:.3f}")

 except Exception as e:
 print(f"\n{model_name}: Error - {str(e)}")
 results[model_name] = None

 return results

# Evaluate models on both datasets
financial_results = evaluate_ml_anomaly_detection(
 X_financial_scaled, financial_df['is_fraud'], ml_models, "Financial Fraud Detection"
)

manufacturing_results = evaluate_ml_anomaly_detection(
 X_manufacturing_scaled, manufacturing_df['is_defective'], ml_models, "Manufacturing Quality Control"
)

# Find best models for each dataset
def find_best_model(results):
 """Find best performing model based on F1 score."""
 valid_results = {k: v for k, v in results.items() if v is not None and v['f1'] is not None}
 if valid_results:
 return max(valid_results.keys(), key=lambda x: valid_results[x]['f1'])
 return None

best_financial_model = find_best_model(financial_results)
best_manufacturing_model = find_best_model(manufacturing_results)

print(f"\n\n BEST PERFORMING MODELS:")
print("=" * 26)
if best_financial_model:
 print(f"• Financial: {best_financial_model} (F1: {financial_results[best_financial_model]['f1']:.3f})")
if best_manufacturing_model:
 print(f"• Manufacturing: {best_manufacturing_model} (F1: {manufacturing_results[best_manufacturing_model]['f1']:.3f})")

In [ ]:
# 3. ENSEMBLE ANOMALY DETECTION
print(" 3. ENSEMBLE ANOMALY DETECTION")
print("=" * 31)

class EnsembleAnomalyDetector:
 """Ensemble anomaly detector combining multiple methods."""

 def __init__(self, models=None, voting='majority', weights=None):
 self.models = models or [
 IsolationForest(contamination=0.1, random_state=42),
 OneClassSVM(gamma='scale', nu=0.1),
 EllipticEnvelope(contamination=0.1)
 ]
 self.voting = voting
 self.weights = weights or [1.0] * len(self.models)
 self.fitted_models = []

 def fit(self, X):
 """Fit all models in the ensemble."""
 self.fitted_models = []
 for model in self.models:
 fitted_model = model.fit(X)
 self.fitted_models.append(fitted_model)
 return self

 def predict(self, X):
 """Predict anomalies using ensemble voting."""
 predictions = []

 for model in self.fitted_models:
 pred = model.predict(X)
 # Convert to binary (1 for normal, 0 for anomaly)
 pred_binary = (pred == 1).astype(int)
 predictions.append(pred_binary)

 predictions = np.array(predictions)

 if self.voting == 'majority':
 # Majority vote (anomaly if majority says anomaly)
 ensemble_pred = (np.mean(predictions, axis=0) < 0.5).astype(int)
 elif self.voting == 'weighted':
 # Weighted vote
 weighted_pred = np.average(predictions, axis=0, weights=self.weights)
 ensemble_pred = (weighted_pred < 0.5).astype(int)

 return ensemble_pred

 def get_anomaly_scores(self, X):
 """Get ensemble anomaly scores."""
 scores = []

 for model in self.fitted_models:
 if hasattr(model, 'decision_function'):
 score = -model.decision_function(X) # Negative for anomaly scores
 elif hasattr(model, 'score_samples'):
 score = -model.score_samples(X)
 else:
 # Fallback to prediction confidence
 pred = model.predict(X)
 score = (pred == -1).astype(float)
 scores.append(score)

 # Average scores
 ensemble_scores = np.mean(scores, axis=0)
 return ensemble_scores

# Test ensemble detector on financial data
print("Financial Data Ensemble Detection:")

ensemble_detector = EnsembleAnomalyDetector(voting='majority')
ensemble_detector.fit(X_financial_scaled)
ensemble_predictions = ensemble_detector.predict(X_financial_scaled)
ensemble_scores = ensemble_detector.get_anomaly_scores(X_financial_scaled)

# Evaluate ensemble performance
ensemble_anomalies = ensemble_predictions == 1
true_positives = ((ensemble_anomalies) & (financial_df['is_fraud'] == 1)).sum()
false_positives = ((ensemble_anomalies) & (financial_df['is_fraud'] == 0)).sum()
false_negatives = ((~ensemble_anomalies) & (financial_df['is_fraud'] == 1)).sum()

ensemble_precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
ensemble_recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
ensemble_f1 = 2 * (ensemble_precision * ensemble_recall) / (ensemble_precision + ensemble_recall) if (ensemble_precision + ensemble_recall) > 0 else 0

print(f"• Ensemble Anomalies detected: {ensemble_anomalies.sum()} ({ensemble_anomalies.mean()*100:.1f}%)")
print(f"• Ensemble Precision: {ensemble_precision:.3f}")
print(f"• Ensemble Recall: {ensemble_recall:.3f}")
print(f"• Ensemble F1-Score: {ensemble_f1:.3f}")

In [ ]:
# 4. COMPREHENSIVE ANOMALY DETECTION DASHBOARD
print(" 4. COMPREHENSIVE ANOMALY DETECTION DASHBOARD")
print("=" * 49)

# Create comprehensive anomaly detection dashboard
fig = make_subplots(
 rows=2, cols=3,
 subplot_titles=[
 'Financial Transaction Anomalies',
 'Manufacturing Quality Control',
 'Statistical vs ML Methods',
 'Anomaly Score Distribution',
 'Model Performance Comparison',
 'ROC Curves'
 ],
 specs=[[{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}],
 [{"secondary_y": False}, {"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Financial Transaction Anomalies (Scatter plot)
fraud_mask = financial_df['is_fraud'] == 1
normal_mask = financial_df['is_fraud'] == 0

fig.add_trace(
 go.Scatter(
 x=financial_df[normal_mask]['transaction_amount'],
 y=financial_df[normal_mask]['account_age_days'],
 mode='markers',
 name='Normal Transactions',
 marker=dict(color='blue', size=4, opacity=0.6)
 ),
 row=1, col=1
)

fig.add_trace(
 go.Scatter(
 x=financial_df[fraud_mask]['transaction_amount'],
 y=financial_df[fraud_mask]['account_age_days'],
 mode='markers',
 name='Fraudulent Transactions',
 marker=dict(color='red', size=6, symbol='x')
 ),
 row=1, col=1
)

# 2. Manufacturing Quality Control
defective_mask = manufacturing_df['is_defective'] == 1
normal_parts_mask = manufacturing_df['is_defective'] == 0

fig.add_trace(
 go.Scatter(
 x=manufacturing_df[normal_parts_mask]['temperature'],
 y=manufacturing_df[normal_parts_mask]['pressure'],
 mode='markers',
 name='Normal Parts',
 marker=dict(color='green', size=4, opacity=0.6)
 ),
 row=1, col=2
)

fig.add_trace(
 go.Scatter(
 x=manufacturing_df[defective_mask]['temperature'],
 y=manufacturing_df[defective_mask]['pressure'],
 mode='markers',
 name='Defective Parts',
 marker=dict(color='red', size=6, symbol='x')
 ),
 row=1, col=2
)

# 3. Statistical vs ML Methods Comparison
methods = []
f1_scores = []

# Add statistical methods
for method, results in stat_results.items():
 if 'f1' in results:
 methods.append(f"Stat: {method}")
 f1_scores.append(results['f1'])

# Add ML methods
for method, results in financial_results.items():
 if results and results['f1'] is not None:
 methods.append(f"ML: {method}")
 f1_scores.append(results['f1'])

fig.add_trace(
 go.Bar(
 x=methods,
 y=f1_scores,
 name='F1 Scores',
 marker_color='lightcoral'
 ),
 row=1, col=3
)

# 4. Anomaly Score Distribution
if best_financial_model and financial_results[best_financial_model]:
 # Get scores from the best model
 best_model = financial_results[best_financial_model]['model']
 if hasattr(best_model, 'decision_function'):
 all_scores = -best_model.decision_function(X_financial_scaled)
 elif hasattr(best_model, 'score_samples'):
 all_scores = -best_model.score_samples(X_financial_scaled)
 else:
 all_scores = ensemble_scores

 normal_scores = all_scores[financial_df['is_fraud'] == 0]
 anomaly_scores = all_scores[financial_df['is_fraud'] == 1]

 fig.add_trace(
 go.Histogram(
 x=normal_scores,
 name='Normal Scores',
 opacity=0.7,
 marker_color='blue',
 nbinsx=30
 ),
 row=2, col=1
 )

 fig.add_trace(
 go.Histogram(
 x=anomaly_scores,
 name='Anomaly Scores',
 opacity=0.7,
 marker_color='red',
 nbinsx=30
 ),
 row=2, col=1
 )

# 5. Model Performance Comparison
model_names = []
model_f1_scores = []

for method, results in financial_results.items():
 if results and results['f1'] is not None:
 model_names.append(method)
 model_f1_scores.append(results['f1'])

fig.add_trace(
 go.Bar(
 x=model_names,
 y=model_f1_scores,
 name='Model F1 Scores',
 marker_color='lightgreen'
 ),
 row=2, col=2
)

# 6. ROC Curve
if best_financial_model and financial_results[best_financial_model]:
 y_true = financial_df['is_fraud']
 try:
 fpr, tpr, _ = roc_curve(y_true, all_scores)

 fig.add_trace(
 go.Scatter(
 x=fpr,
 y=tpr,
 mode='lines',
 name=f'ROC ({best_financial_model})',
 line=dict(color='purple', width=2)
 ),
 row=2, col=3
 )

 # Add diagonal reference line
 fig.add_trace(
 go.Scatter(
 x=[0, 1],
 y=[0, 1],
 mode='lines',
 name='Random Classifier',
 line=dict(color='gray', width=1, dash='dash')
 ),
 row=2, col=3
 )
 except:
 pass

# Update layout
fig.update_layout(
 height=800,
 title="Comprehensive Anomaly Detection Dashboard",
 showlegend=True
)

# Update axis labels
fig.update_xaxes(title_text="Transaction Amount", row=1, col=1)
fig.update_xaxes(title_text="Temperature", row=1, col=2)
fig.update_xaxes(title_text="Method", row=1, col=3)
fig.update_xaxes(title_text="Anomaly Score", row=2, col=1)
fig.update_xaxes(title_text="Model", row=2, col=2)
fig.update_xaxes(title_text="False Positive Rate", row=2, col=3)

fig.update_yaxes(title_text="Account Age", row=1, col=1)
fig.update_yaxes(title_text="Pressure", row=1, col=2)
fig.update_yaxes(title_text="F1 Score", row=1, col=3)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="F1 Score", row=2, col=2)
fig.update_yaxes(title_text="True Positive Rate", row=2, col=3)

fig.show()